<a href="https://colab.research.google.com/github/udanish50/ChebyShevHyperNet/blob/main/ECE9309_9039_Exam2023_Part1_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Exam – Part 1 (60 pts)

### Student Name: xxxxxxxx
### Student ID: xxxxxxxx

## General
The instructions for the final exam are included in the cover page shared on OWL. In addion, some basic rules:

This part of the exam is **Open Book** and you will answer to the programming questions below on this Jupyter Notebook. You have **2 hours (9:00 am - 11:00 am)** to finish the exam and upload your notebook on OWL. 
* You **are allowed** to use any document and sources on your computer and look up documents on the internet. **You need to cite any code that you use if it is NOT from the course Labs or Tutorial examples**.
* You or **NOT allowed** to share documents, or communicate in any other way with people about the final during the final. Afterwards, you are not allowed to share the final, or talk about its content, with students who still have to take a make-up final. Using AI chatbots is **NOT allowed and will be counted as cheating or plagiarism**.
* Many questions also require a written answer. The answer to these questions should be given in full English sentences.
* All Figures should have a x-axis and y-axis label.
* Add as many cells as you want, whenever you need to. 
* Please make sure to add sufficient comments to your code.
* To finish the exam in the alloted time, you will have to work efficiently. **Read the requirements of each question carefully.**
* You need to submit the final exam Jupyter Notebook by the **due date (April 17, 2023 at 11:00 am)** on **OWL in the Assignments / Final Exam - Part 1** where you downloaded the Dataset and Jupyter Notebook. Late submission will be scored with 0 pts, unless you have received special accommodations. To avoid technical difficulties, start your submission at latest five to ten minutes before the deadline. To be sure, you can also submit multiple versions - only the latest version will be graded. 


**Please remember that you also have Part 2 of this exam (40 Points) that will be released on OWL at 11:00am and closed at 12:00pm.**

**Ensure that your code runs correctly by choosing "Kernel -> Restart and Run All" before submitting.**

### Additional Guidance

If at any point you are not sure about the answer, then *write your assumptions clearly in your exam and proceed according to those assumptions.*

Good luck!

In [1]:
## Preliminaries
# Sets up the environment by importing 
# pandas, numpy, matplotlib.

### YOU MAY ADD ADDITIONAL IMPORTS IF YOU WISH
### HERE OR IN ANY OF THE CELLS BELOW

import matplotlib.pyplot as plt 
%matplotlib inline

import pandas as pd 
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, confusion_matrix, roc_curve, auc, roc_auc_score

import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


---
---
# Question 1 - Data Preparations ( X / 5 pts )

## Dataset
In this exam, we will work on the network anomaly detection dataset "IP_Activity_Dataset_5000.csv". It was generated from web server access logs and collected from a real-world website in Content Delivery Networks (CDNs). Each sample/row in the dataset represents a unique Internet Protocol (IP) address with 9 columns/variables. Each feature/column is a performance indicator that reflect the state or activity of each sample/IP. The IP addresses were masked due to privacy reasons.  

### Variables/Features
Feature description: 
1.	**requests**: the number of requests sent by per IP.
2.	**request-interval**:  the average time interval between consecutive requests sent by per IP. Unit: milliseconds
3.	**request-popularity**: what percentage of the requests sent by per IP are for popular contents.
4.	**bytes**: the average bytes received by per IP after requesting the content.
5.	**delivery-time**: the average request delivery time experienced by per IP. Unit: milliseconds
6.	**hit-rate**: cache hit rate of per IP.
7.	**nodes**: the number of nodes that received requests from per IP.
8.	**contents**: the number of contents/files that per IP requested for.
9.	**label**: 0-normal, 1-abnormal (potential cache pollution attacks). 

---
## Question 1 Part A - Read Dataset ( X / 1 pts )
Read the dataset "IP_Activity_Dataset_5000.csv" as a pandas dataframe "df". PS: Please use the relative path.

In [4]:
df = pd.read_csv('IP_Activity_Dataset_5000.csv')

---
## Question 1 Part B - Explore dataset ( X / 4 pts )
In the cell below, write code that does the following:
- Prints the number of observations in the dataset
- Prints the number of variables in the dataset (all variables regardless of whether they are a feature or label or neither)
- Prints the number of observations for each class in the 'label' variable
- Prints the first five rows of the dataset

In [8]:
print(df.shape)
print(df[['label']].value_counts())
df.head()

(5000, 9)
label
0        4500
1         500
dtype: int64


,requests,request-interval,request-popularity,bytes,delivery-time,hit-rate,nodes,contents,label
0,18,378.605278,1.000000,29.667209,38.833333,0.888889,2,9,0
1,5,0.524200,1.000000,19.226953,204.600000,1.000000,1,5,0
2,1141,66.912231,0.167647,19.847204,142.072963,0.021520,5,703,1
3,80,2.472875,0.418750,1769.312634,1330.787500,1.000000,2,76,0
4,1,0.000000,1.000000,0.000000,2.000000,1.000000,1,1,0


---
---
# Question 2 - Regression and Evaluation (X / 20 pts)
Your next task is to build regression models that predicts the delivery-time of IPs.

In [10]:

model_LR = LinearRegression()
df.columns

Index(['requests', 'request-interval', 'request-popularity', 'bytes',
       'delivery-time', 'hit-rate', 'nodes', 'contents', 'label'],
      dtype='object')

---
## Question 2 Part A - Data Splitting For Regression ( X / 2 pts )
- Use 'delivery-time' as the target variable y for your regression models, and other variables as the feature set X.
- Split the data into equals-sized training and test sets (do not shuffle the data).


In [15]:
features = df[['requests', 'request-interval', 'request-popularity', 'bytes','hit-rate', 'nodes', 'contents', 'label']]
target = df[['delivery-time']]
Xtrain, Xtest, ytrain, ytest = train_test_split(features, target,test_size=0.5, shuffle=False)

---
## Question 2 Part B - Data Standardization ( X / 2 pts )
- Z-standarize the input features of the training and test sets.
- All the questions below should be based on the standarized dataset.

In [16]:
scaler = StandardScaler()
XtrainScaled = scaler.fit_transform(Xtrain)
XtestScaled = scaler.fit_transform(Xtest)

---
## Question 2 Part C - Basic Lasso Regression ( X / 2 pts )
- Build a regression model with L1 regularization (Lasso) and the default alpha value. Fit it on your training set, and set the random state to 42.
- Report the coefficients and intercept of the model.
- Report the Root Mean Square Error (RMSE) to evaluate the testing performance of your model.

In [19]:
Model_Lasso = Lasso()
Model_Lasso.fit(XtrainScaled, ytrain)

#Report the coefficients and intercept of the model.
print('oefficients:', Model_Lasso.coef_)
print('Intercept:', Model_Lasso.intercept_)

# predictions
ypred = Model_Lasso.predict(XtestScaled)

# Root Mean Square Error (RMSE) to evaluate the testing performance of your model.
rmse = mean_squared_error(ytest, ypred)**0.5
print('Root Mean Square Error:', rmse)


oefficients: [  10.17733701   18.59594951  103.74664627 5222.06671709 -138.53886518
  -26.9272339   328.28445291 -528.78077484]
Intercept: [522.28555137]
Root Mean Square Error: 4572.98520683828


---
## Question 2 Part D - Determine the Optimal Regularization Term ( X / 5 pts )
- Perform Lasso Regression with 5-fold cross-validation on the training set to find and **print out** the optimal regularization parameter (alpha) value. Vary the regularization parameter (alpha) between 0.01 and 100, evenly spaced in log-space, and generate 100 values. Set the random state to 42. Tip: use LassoCV function.
- Create a plot showing the relationship between these 100 alpha values and their corresponding mean RMSE values. Sets the scale of the x-axis to a logarithmic scale. 
- Build and fit a Lasso Regression model on the training set using the optimal alpha and a random state of 42. Report the coefficients and intercept of the model. Report the Root Mean Square Error (RMSE) to evaluate the testing performance of your model.

In [25]:
regularization_strength = np.exp(np.linspace(np.log(0.2),np.log(250),50))

---
## Question 2 Part E - Identify Important Features Using Lasso Regression ( X / 5 pts )
- Build Lasso Regression models on a dataset with n features (n = 1, 2, ..., 8) and plots the Root Mean Squared Error (RMSE) against the number of features used. Please follow the following steps:
    - Determine feature importance using the coefficients of the optimized Lasso model from Q-2D. Tip: The features with larger absolute coefficients are considered more important.
    - Iteratively fit the optimized Lasso model with different numbers of features (top-n most important features) (n = 1, 2, ..., 8).
    - Calculate and store RMSE values for each iteration.
    - Print out the names of the features in order of importance, from most to least important.
    - Plot the RMSE values against the number of important features used (from 1 - 8). Add appropriate x-axis and y-axis labels.

---
## Question 2 Part F - Discussion: Feature Selection Using Lasso Regression ( X / 4 pts )
Based on the plot from Q2-E, answer the following two questions in text:
- A) Determine which feature is the most important feature and discuss the reasons why this feature is the most important feature to predict delivery-time.
- B) Determine how many features should be selected, give their names, and discuss the reasons, based on the plot from Q2-F.

#### A) Written answer: Explain here.



#### B) Written answer: Explain here.


---
---
# Question 3 - Classification and Evaluation (X / 35 pts)
Your next task is to build classification models that can identify the malicious attacker IPs.

---
## Question 3 Part A - Data Splitting For Classification ( X / 1 pts )
- Use 'label' as the target variable y for your classification models for abnormal IP detection, and other variables as the feature set X.
- Split the data into equals-sized training and test sets (do not shuffle the data).

In [23]:
features = df[['requests', 'request-interval', 'request-popularity', 'bytes','delivery-time','hit-rate', 'nodes', 'contents']]
target = df[['label']]
X_train, X_test, y_train, y_test = train_test_split(features, target,test_size=0.5, shuffle=False)

---
## Question 3 Part B - Data Standardization ( X / 1 pts )
- Z-standarize the input features of the training and test sets.
- All the questions below should be based on the standarized dataset.

In [24]:
scaler = StandardScaler()
X_trainScaled = scaler.fit_transform(X_train)
X_testScaled = scaler.fit_transform(X_test)

---
## Question 3 Part C - Random Forest ( X / 3 pts )
- Build a Random Forest model that consists of 5 base decision trees with the maximum depth of 5, and fit the training set. Set random state to 42.
- Print out the accuracy, F1-score, confusion matrix, and execution time (including both training and testing time) of the model when evaluating the testing performance of your model.


In [ ]:
  #Build a Random Forest model that consists of 5 base decision trees with the maximum depth of 5
  myRandomForestModel = RandomForestClassifier(n_estimators=10, random_state=42)
  myRandomForestModel.fit(Xtrain, ytrain)

---
## Question 3 Part D - Feature Selection by Random Forest ( X / 10 pts )
- Use the Random Forest model you built in Q3-C to generate feature importance scores and select the most important features (rank the importance scores of each feature in descending order, and only select the important features from most to least important until the accumulated relative importance score reaches 90% or 0.9).
- Use a horizontal bar chart to plot the importance scores of all features in descending order. Add appropriate x-axis and y-axis labels.
- Print out the selected features with their importance scores, and generate the new training and test sets with the new feature set. 
- Retrain the same Random Forest model from Q3-C on the new training set, and print out the accuracy, F1-score, confusion matrix, and execution time (including both training and testing time) of the model on the new test set.

---
## Question 3 Part E - Hyperparamete Tuning of Random Forest ( X / 5 pts )
- Use 3-fold grid search to tune two hyperparameters for the Random Forest model you built in Q3-D:
    - The number of base estimators/decision trees (find the better value among the two numbers 10 and 20).
    - The maximum tree depth (find the better value among the two numbers 10 and 20).
- Print out the detected better hyperparameter values and cross-validation score.
- Build the Random Forest model with the better hyperparameter values you found, and fit the new training set from Q3-D.
- Report the accuracy, F1-score. confusion matrix, and execution time (including both training and testing time) of the model when evaluating the testing performance of your model on the new test set from Q3-D.


---
## Question 3 Part F - Classification Model Performance Discussion ( X / 5 pts )
- Compare the performance of the three models from Question 3-C, 3-D, and 3-E, and discuss reasons for performance difference.
- Compare the execution time of the three models from Question 3-C, 3-D, and 3-E, and discuss reasons for time/efficiency difference.

#### Written answer: Explain here.

#### Written answer: Explain here.

---
## Question 3 Part G - MLP Model ( X / 5 pts )
Build a MLP model to fit the training set from Question 3-B. The details of the MLP model are as follows:
- The model should have two hidden layers with 32 neurons each, and use ReLU activation function. 
- The output layer should have sigmoid activation function. 
- Compile the model with binary cross-entropy loss function, Adam optimizer with learning rate of 0.001, and accuracy as the metric.
- Train the model for 10 epochs with batch size of 32. 
- Evaluate the trained model on the test data, and print the accuracy, F1-score. confusion matrix, and execution time (including both training and testing time) of the model.

---
## Question 3 Part H - MLP Loss Change Plot ( X / 2 pts )
Record the loss for each iteration of the MLP model, and make a plot of the number of iterations/epochs vs loss(Binary Cross Entropy). Add appropriate x-axis and y-axis labels.

---
## Question 3 Part I - ROC Curve Plot ( X / 3 pts )
- Plot the ROC curve for evaluating the model on the test set.
- Report the area under the ROC curve.

---
---
---
**You're done! As always, double-check your work by re-running the notebook from scratch.**